In [9]:
import dalex as dx
import pickle as pkl
import data_prep
import pandas as pd
import plotly.graph_objects as go

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv("marketing_campaign.csv", sep = ";")
X = data.drop("Response", axis=1)
y = data["Response"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 2115)

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
class YearFromDtTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Assuming X is a DataFrame and "Dt_Customer" is one of its columns
        year_column = X["Dt_Customer"].str[:4].astype(int)
        return year_column.values.reshape(-1, 1)

In [13]:
# rf = pkl.load(open("models/rf1.pkl", 'rb'))
xgb = pkl.load(open("models/xgb_v2.pkl", 'rb'))

In [29]:
df = pd.read_csv("marketing_campaign.csv", sep=";")

x_train, y_train, x_test, y_test = data_prep.prep_data(df)

In [14]:
# exp = dx.Explainer(rf, x_train, y_train, model_type="classification")
exp = dx.Explainer(xgb, X_train, y_train, model_type="classification")

Preparation of a new explainer is initiated

  -> data              : 1792 rows 28 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 1792 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x0000027B055EF880> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 2.4e-05, mean = 0.147, max = 0.994
  -> model type        : classification will be used
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.525, mean = 6.92e-05, max = 0.768
  -> model_info        : package imblearn

A new explainer has been created!


In [54]:
exp.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.942966,0.984127,0.963107,0.989397,0.999478


In [55]:
mp = exp.model_parts()

In [56]:
vip = mp.result
vip.sort_values(by="dropout_loss", ascending=False)
exclude_values = ["_baseline_","_full_model_"]
vip = vip.query('variable not in @exclude_values')
vip

,variable,dropout_loss,label
1,AcceptedCmp2,0.000533,XGBClassifier
2,Z_Revenue,0.000533,XGBClassifier
3,Z_CostContact,0.000533,XGBClassifier
4,Complain,0.000533,XGBClassifier
5,NumDealsPurchases,0.001324,XGBClassifier
6,NumWebPurchases,0.001780,XGBClassifier
7,Kidhome,0.001808,XGBClassifier
8,AcceptedCmp4,0.002083,XGBClassifier
9,Year_Birth,0.002438,XGBClassifier
10,Education,0.002632,XGBClassifier


In [57]:
mp.plot()

In [16]:
pd_rf = exp.model_profile(variables=["MntMeatProducts","Recency","Marital_Status"])

Calculating ceteris paribus:   0%|          | 0/3 [00:00<?, ?it/s]d:\Programy\miniconda3\envs\bhl9\lib\site-packages\dalex\predict_explanations\_ceteris_paribus\utils.py:104: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[   0.     17.25   34.5  ... 1690.5  1707.75 1725.  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_data.loc[:, variable] = np.tile(split_points, data.shape[0])
Calculating ceteris paribus:  33%|███▎      | 1/3 [00:00<00:00,  7.93it/s]d:\Programy\miniconda3\envs\bhl9\lib\site-packages\dalex\predict_explanations\_ceteris_paribus\utils.py:104: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.    0.99  1.98 ... 97.02 98.01 99.  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_data.loc[:, variable] = np.tile(split_points, data.shape[0])


In [39]:
pd_rf = exp.model_profile(variables=["Marital_Status"],variable_type="categorical")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 34.60it/s]


In [23]:
pd_rf.plot()

In [31]:
pd_rf.mean_prediction

0.16062571

In [40]:
pd_rf.result

,_vname_,_label_,_x_,_yhat_,_ids_
0,Marital_Status,XGBClassifier,Single,0.196404,0
1,Marital_Status,XGBClassifier,Widow,0.161578,0
2,Marital_Status,XGBClassifier,Divorced,0.170683,0
3,Marital_Status,XGBClassifier,Married,0.120944,0
4,Marital_Status,XGBClassifier,Together,0.117419,0
5,Marital_Status,XGBClassifier,YOLO,0.161578,0
6,Marital_Status,XGBClassifier,Alone,0.161578,0
7,Marital_Status,XGBClassifier,Absurd,0.161578,0


In [37]:
pd_rf["_yhat_"].sum()/8

0.1766701489686966

In [61]:
pd_rf[pd_rf["_vname_"]=="Recency"]

,_vname_,_label_,_x_,_yhat_,_ids_
101,Recency,XGBClassifier,0.00,0.331628,0
102,Recency,XGBClassifier,0.99,0.331628,0
103,Recency,XGBClassifier,1.98,0.331628,0
104,Recency,XGBClassifier,2.97,0.331628,0
105,Recency,XGBClassifier,3.96,0.331628,0
...,...,...,...,...,...
197,Recency,XGBClassifier,95.04,0.065664,0
198,Recency,XGBClassifier,96.03,0.065664,0
199,Recency,XGBClassifier,97.02,0.065664,0
200,Recency,XGBClassifier,98.01,0.065664,0


In [62]:
colors = ['blue' if val < max(pd_rf["_yhat_"])-0.1*max(pd_rf["_yhat_"]) else 'green' for val in pd_rf['_yhat_']]

In [63]:
fig = go.Figure()

for i in range(len(df) - 1):
    fig.add_trace(go.Scatter(
        x=df['_x_'].iloc[i:i+2],
        y=df['_yhat_'].iloc[i:i+2],
        mode='lines',
        line=dict(color=colors[i]),
        showlegend=False
    ))

fig.update_layout(
    title='Line Plot',
    xaxis=dict(title='Wartość'),
    yaxis=dict(title='Prawdopodobieństwo')
)

KeyError: '_x_'

In [15]:
pd_rf.plot()